In [ ]:
install.packages("pacman")

pacman::p_load(
    ISLR,       #For the dataset Carseats
    #GGally,    #For scatterplotmatrix
    magrittr,   # Pipes
    caret,      #Facilitates hyperparameter tuning with cross-validation
    rpart.plot,
    tidymodels,
    ggplot2,
    tidyr,
    rlang,
    lmtest
)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


ISLR installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘Rcpp’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Warning message in utils::install.packages(package, ...):
“installation of package ‘pROC’ had non-zero exit status”
Warning message in utils::install.packages(package, ...):
“installation of package ‘reshape2’ had non-zero exit status”
Warning message in utils::install.packages(package, ...):
“installation of package ‘lava’ had non-zero exit status”
Warning message in utils::install.packages(package, ...)

In [ ]:
#remove.packages("rlang")  # Remove the current version
#install.packages("rlang")  # Install the latest version
#install.packages("lmtest")

#library(rlang)  # Load the rlang package

# Load your other packages
#library(ISLR)
#library(magrittr)
#library(caret)
#library(rpart.plot)
#library(tidymodels)
#library(ggplot2)

In [ ]:
class(Carseats)

In [ ]:
summary(Carseats)

In [ ]:
head(Carseats)

In [ ]:
tail(Carseats)

In [ ]:
sum(is.na(Carseats))

# Create a histogram for all numeric variables in one plot

In [ ]:
# Prepare the dataset for ggplot2
carseats_data_long <- Carseats %>%
    select(Sales, CompPrice, Income, Advertising, Population, Price, Age, Education) %>%
       pivot_longer(cols = everything(),
              names_to = "variable",
              values_to = "value")

# Create a histogram for all numeric variables in one plot
carseats_histograms <- ggplot(carseats_data_long, aes(x = value)) +
 geom_histogram(bins = 30, color = "black", fill = "lightblue") +
 facet_wrap(~variable, scales = "free", ncol = 4) +
 labs(title = "Histograms of Numeric Variables in the Sales of Child Car Seats dataset",
      x = "Value",
      y = "Frequency") +
 theme_minimal()

# Plot the histograms
print(carseats_histograms)

In [ ]:
pairs(~Sales + CompPrice + Income + Advertising + Population + Price + Age + Education, data = Carseats,lower.panel = panel.smooth)

# Creating the Train / Test set

In [ ]:
# Split the data into training and testing sets

set.seed(123)

#data_split <- initial_split(Carseats, prop = 0.80)
#train_data <- training(data_split)

data <- Carseats
trainIndex <- createDataPartition(data$Price, p = 0.7, list = FALSE)
train_data <- data[trainIndex, ] %>%
  glimpse()

In [ ]:
#test_data <- testing(data_split) %>%
test_data <- data[-trainIndex, ] %>%
  glimpse()

# Define the hyperparameter grid

In [ ]:
hyperparameter_grid <- expand.grid(
  cp = seq(0.001, 0.1, 0.001)     # Complexity parameter (pruning parameter)
  #minsplit = c(2, 5, 10),        # Minimum number of samples required to split a node
  #minbucket = c(1, 5, 10),       # Minimum number of samples required for a terminal node (leaf)
  #maxdepth = c(5, 10, 15)        # Maximum depth of the tree
)

# Perform cross-validation and select the best hyperparameters

In [ ]:
ctrl <- trainControl(method = "cv", number = 5)  # 5-fold cross-validation
model <- train(Price ~ .,
               data = train_data,
               method = "rpart",
               trControl = ctrl,
               tuneGrid = hyperparameter_grid)

# Train the final model with the best **hyperparameters**

In [ ]:
best_hyperparameters <- model$bestTune
final_model <- rpart(Price ~ ., data = train_data,
                     cp = best_hyperparameters$cp)
                     #minsplit = best_hyperparameters$minsplit,
                     #minbucket = best_hyperparameters$minbucket,
                     #maxdepth = best_hyperparameters$maxdepth)

# Create a decision tree model specification

In [ ]:

#tree_spec <- decision_tree() %>%
# set_engine("rpart") %>%
# set_mode("regression")

# Fit the model to the training data
#tree_fit <- tree_spec %>%
# fit(Price ~ ., data = train_data)

# Make predictions on the testing data

In [ ]:
#predictions <- tree_fit %>%
# predict(test_data) %>%
# pull(.pred)

predictions <- predict(final_model, newdata = test_data)

# Calculate RMSE and R-squared
metrics <- metric_set(rmse, rsq)
model_performance <- test_data %>%
 mutate(predictions = predictions) %>%
 metrics(truth = Price, estimate = predictions)

print(model_performance)

In [ ]:
# Make predictions on new data
new_data <- tribble(
 ~Sales, ~CompPrice, ~Income, ~Advertising, ~Population, ~ShelveLoc, ~Age, ~Education, ~Urban, ~US,
 9.50, 138, 73, 11, 276, 'Bad', 42, 17, 'Yes', 'Yes'
)
predictions <- predict(final_model, new_data)
print(predictions)

#Análisis de donde mejorar el shelve location,


In [ ]:
data("Carseats")  # Load the Carseats dataset

# Step 3: Create a frequency table for the ShelveLoc column
freq_table <- table(Carseats$ShelveLoc)

# Step 4: Create the bar chart using ggplot
ggplot(data.frame(ShelveLoc = names(freq_table), Count = as.vector(freq_table)), aes(x = ShelveLoc, y = Count)) +
  geom_bar(stat = "identity", fill = "blue", color = "black") +
  labs(title = "Count per Shelve Location",
       x = "Shelve Location",
       y = "Count") +
  theme_minimal()

#Modelo Precio vs. CompPrice usando Regersión Lineal

In [ ]:
# Assuming you have a data frame called "df" with the same structure as the current dataset

# Load the necessary library
library(lmtest)

df <- Carseats

# Split the data into a training set and a test set
set.seed(123)  # for reproducibility
train_indices <- sample(1:nrow(df), nrow(df)*0.8)
train_df <- df[train_indices, ]
test_df <- df[-train_indices, ]

# Fit a linear regression model
model <- lm(Price ~ CompPrice, data = train_df)

# Print the summary of the model
summary(model)

# Predict on the test set
predictions <- predict(model, newdata = test_df)

# Evaluate the model
rmse <- sqrt(mean((predictions - test_df$Price)^2))
print(paste0("Root Mean Squared Error (RMSE): ", rmse))

In [ ]:
# Load necessary library
library(lmtest)

# Fit a linear regression model with Sales as the dependent variable and Advertising as the independent variable
model <- lm(Sales ~ Advertising, data = df)

# Print summary of the model to get coefficients
summary(model)

# Predict Sales based on the Advertising budget
predictions <- predict(model, newdata = df)

# Calculate residuals
residuals <- df$Sales - predictions
#A big F, with a small p-value, means that the null hypothesis is discredited,
#and we would assert that there is a general relationship between the response and predictors (while a small F, with a big p-value indicates that there is no relationship).

# Plot residuals to check for any pattern (there should ideally be no pattern in the residuals)
plot(residuals)

In [ ]:
# Load the necessary packages
library(rpart)

# Fit a decision tree model
model <- rpart(Sales ~ Advertising, data = train_data, method = "anova")

# Print the summary of the model
printcp(model)

# Plot the decision tree
plot(model, uniform = TRUE, main = "Decision Tree for Sales vs Advertising")
text(model, use.n = TRUE, all = TRUE, cex = .8)

# Predict on the same data (or new data)
predictions <- predict(model, newdata = test_data)

mse <- mean((test_data$Sales - predictions)^2)
print(paste("testmse", mse))


#Random Forest to Improve ShelveLocation column

In [ ]:
# Load required libraries
library(ISLR)
library(randomForest)
library(rpart.plot)

# Load the Carseats dataset
data("Carseats")

# Explore the structure of the dataset
str(Carseats)

# Assuming "ShelveLoc" is a categorical variable we want to predict
# Let's convert it to a factor for classification
Carseats$ShelveLoc <- as.factor(Carseats$ShelveLoc)

# Split the data into training and testing sets
set.seed(42) # For reproducibility
train_indices <- sample(1:nrow(Carseats), 0.7 * nrow(Carseats))
train_data <- Carseats[train_indices, ]
test_data <- Carseats[-train_indices, ]

# Create a Random Forest model
rf_model <- randomForest(ShelveLoc ~ ., data = train_data, ntree = 100)

# Print the model summary to see important statistics
print(rf_model)



In [ ]:
# Plot the decision tree for visualization
tree <- getTree(rf_model, k = 1, labelVar = TRUE)
rpart.plot(tree)

# Make predictions on the test data
rf_predictions <- predict(rf_model, newdata = test_data)

# Model performance evaluation (assuming a classification task)
confusion_matrix <- table(test_data$ShelveLoc, rf_predictions)
accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
cat("Accuracy:", accuracy, "\n")

# You can also view other performance metrics like precision, recall, F1-score, etc.
# depending on the problem and the number of classes.


In [ ]:
# Extract a single tree from the forest for visualization
single_tree <- getTree(rf_model, k = 1, labelVar = TRUE)

# Plot the decision tree for visualization
rpart.plot(single_tree)

#Using Categorical Decision Trees to improve the shelve location

In [ ]:
# Convert ShelveLoc to numeric format
Carseats$ShelveLoc_num <- as.numeric(factor(Carseats$ShelveLoc, levels = c("Bad", "Medium", "Good")))


In [ ]:
# Set seed for reproducibility
set.seed(123)

# Split data into training and testing sets (70% training, 30% testing)
train_index <- sample(1:nrow(Carseats), 0.7 * nrow(Carseats))
train_data <- Carseats[train_index, ]
test_data <- Carseats[-train_index, ]


In [ ]:

# Create the decision tree model
tree_model <- rpart(ShelveLoc_num ~ . - Sales, data = train_data)


In [ ]:
# Plot the decision tree
rpart.plot(tree_model, extra = 101, under = TRUE, tweak = 1, box.palette = "RdBu")


In [ ]:
# Convert numeric predictions to categorical levels
convert_to_levels <- function(predictions) {
  levels <- c("Bad", "Medium", "Good")
  return(levels[round(predictions)])
}

# Make predictions using the decision tree model
predictions <- predict(tree_model, test_data, type = "vector")

# Convert numeric predictions to categorical levels
predicted_levels <- convert_to_levels(predictions)

# Calculate accuracy
accuracy <- mean(predicted_levels == test_data$ShelveLoc)

# Print accuracy
cat("Model Accuracy:", round(accuracy * 100, 2), "%\n")


In [ ]:
install.packages("pacman")

pacman::p_load(
  ISLR
)